In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df_original = pd.read_csv(r'rating_observacoes_2010_2023.csv')

In [ ]:
df = df_original.copy()

In [ ]:
mes_map = {
    "janeiro": "01",
    "fevereiro": "02",
    "março": "03",
    "abril": "04",
    "maio": "05",
    "junho": "06",
    "julho": "07",
    "agosto": "08",
    "setembro": "09",
    "outubro": "10",
    "novembro": "11",
    "dezembro": "12"
}

df['mes'] = df['mes'].replace(mes_map)

In [ ]:
df['mes'] = df['mes'].str.capitalize()  # Garantir nomes consistentes

# Agrupar por ano e mês
df['mes_ano'] = df['ano'].astype(str) + df['mes']

total_por_mes = df.groupby('mes_ano')['num_observacoes'].transform('sum')

df['proporcao'] = df['num_observacoes'] / total_por_mes


In [ ]:
# Calcular o PSI entre dois meses mas temos de especificar o ano, por exemplo janeiro de 2010 com fevereiro de 2010

def calculate_psi_categorical(df, mes_base, mes_comp):
    base = df[df['mes_ano'] == mes_base].set_index('rating')['proporcao']
    comp = df[df['mes_ano'] == mes_comp].set_index('rating')['proporcao']
    
    # Alinhar índices (caso ratings não estejam todos presentes em ambos os meses)
    all_ratings = base.index.union(comp.index)
    base = base.reindex(all_ratings, fill_value=0.0001)
    comp = comp.reindex(all_ratings, fill_value=0.0001)
    
    psi = np.sum((base - comp) * np.log(base / comp))
    return psi

In [ ]:
# Overlapping
# Somar todos os janeiros e fevereiros independentemente do ano, ou seja compara todos os janeiros com todos os fevereiros
jan_total = df[df['mes'] == '01'].groupby('rating')['num_observacoes'].sum()
fev_total = df[df['mes'] == '02'].groupby('rating')['num_observacoes'].sum()

df_jan_fev = pd.concat([
    jan_total.rename('jan'),
    fev_total.rename('fev')
], axis=1).fillna(0.0001)

df_jan_fev['p_jan'] = df_jan_fev['jan'] / df_jan_fev['jan'].sum()
df_jan_fev['p_fev'] = df_jan_fev['fev'] / df_jan_fev['fev'].sum()

psi = np.sum((df_jan_fev['p_jan'] - df_jan_fev['p_fev']) * np.log(df_jan_fev['p_jan'] / df_jan_fev['p_fev']))
print(f"PSI Janeiro vs Fevereiro: {psi:.4f}")


In [ ]:
# Non Overlapping
# Compara janeiro com os outros meses todos
meses = ['02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']

resultados = []

for mes in meses:
    ref = df[df['mes'] == '01'].groupby('rating')['num_observacoes'].sum()
    comp = df[df['mes'] == mes].groupby('rating')['num_observacoes'].sum()
    
    tmp = pd.concat([ref.rename('jan'), comp.rename('outro')], axis=1).fillna(0.0001)
    tmp['p_jan'] = tmp['jan'] / tmp['jan'].sum()
    tmp['p_outro'] = tmp['outro'] / tmp['outro'].sum()
    
    psi_val = np.sum((tmp['p_jan'] - tmp['p_outro']) * np.log(tmp['p_jan'] / tmp['p_outro']))
    resultados.append({'mes_comparado': mes, 'psi': psi_val})

psi_df = pd.DataFrame(resultados)
print(psi_df)


In [ ]:
# Non Overlapping
# Calcular o PSI(jan2010, feb2010), PSI(jan2011, feb2011), ..., PSI(jan2023, feb2023)
#            PSI(jan2010, mar2010), PSI(jan2011, mar2011), ..., PSI(jan2023, mar2023)
#            PSI(jan2010, apr2010), PSI(jan2011, apr2011), ..., PSI(jan2023, apr2023)
#            PSI(jan2010, may2010), PSI(jan2011, may2011), ..., PSI(jan2023, may2023)
#            PSI(jan2010, jun2010), PSI(jan2011, jun2011), ..., PSI(jan2023, jun2023)
#            PSI(jan2010, jul2010), PSI(jan2011, jul2011), ..., PSI(jan2023, jul2023)
#            PSI(jan2010, aug2010), PSI(jan2011, aug2011), ..., PSI(jan2023, aug2023)
#            PSI(jan2010, sep2010), PSI(jan2011, sep2011), ..., PSI(jan2023, sep2023)
#            PSI(jan2010, oct2010), PSI(jan2011, oct2011), ..., PSI(oct2023, oct2023)
#            PSI(jan2010, nov2010), PSI(jan2011, nov2011), ..., PSI(jan2023, nov2023)
#            PSI(jan2010, dec2010), PSI(jan2011, dec2011), ..., PSI(jan2023, dec2023)

# Fazer o mesmo para os outros meses: 
# feb vs all months, 
# mar vs all months, 
# apr vs all months,
# may vs all months,
#...

# Overlapping
# Calcular o PSI(jan, feb)
#            PSI(feb, mar)
#            PSI(mar, apr)
#            PSI(apr, may)
#            PSI(may, jun)
#            PSI(jun, jul)
#            PSI(jul, aug)
#            PSI(aug, sep)
#            PSI(sep, oct)
#            PSI(oct, nov)
#            PSI(nov, dec)